<a href="https://colab.research.google.com/github/Deni1019/Bootcamp-Caracteristicas-del-mercado-energ-tico/blob/main/limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
pip install pandas numpy openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
#importar librerias
import pandas as pd
import os
import re

In [11]:
#Definir las rutas de acceso a los archivos
ruta_demanda = 'data/demanda_energetica'
ruta_produccion = 'data/produccion_energetica'
ruta_diccionario = 'data/produccion_energetica/diccionario_plantas_departamento.xlsx'

In [12]:
diccionario = pd.read_excel(ruta_diccionario)
diccionario.columns = diccionario.iloc[6]
diccionario = diccionario.drop(index = range(0,10))
diccionario = diccionario.reset_index(drop=True)
print("DataFrame original:")
print(diccionario.head())
print('_______________________________')
print('Serie de planta vs dpto.')
clave = diccionario[['Tipo/Fuente de energía','Departamento']]
print(clave.head())
print('_______________________________')
print('Serie filtrada')
mascara = clave['Departamento'].str.strip() != '-'
clave_filtrada = clave[mascara]
print(clave_filtrada.head())
print(f'Tamaño de la serie filtrada: {clave_filtrada.shape}')

DataFrame original:
6 Tipo/Fuente de energía Capacidad efectiva neta [MW]  \
0                 AMALFI                         0,81   
1                AMERICA                         0,41   
2                 ASNAZU                         0,45   
3                 BAYONA                          0,6   
4                  BELLO                         0,35   

6                              Operador Fecha de puesta en operación FPO  \
0               HZ ENERGY S.A.S. E.S.P.                       05/08/2007   
1  EMPRESAS PUBLICAS DE MEDELLIN E.S.P.                       01/01/1997   
2                     VATIA S.A. E.S.P.                       01/01/1934   
3      EMPRESAS PUBLICAS DE CALARCA ESP                       01/01/1943   
4  EMPRESAS PUBLICAS DE MEDELLIN E.S.P.                       01/01/1997   

6 Municipio Departamento            Subárea  
0    AMALFI    ANTIOQUIA  SubArea Antioquia  
1  MEDELLÍN    ANTIOQUIA  SubArea Antioquia  
2    SUÁREZ        CAUCA      SubArea Vall

In [13]:
clave_filtrada.rename(columns={'Tipo/Fuente de energía': 'Planta'}, inplace=True)
diccionario_plantas = clave_filtrada
diccionario_plantas.head()

/tmp/ipykernel_15503/3767197889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clave_filtrada.rename(columns={'Tipo/Fuente de energía': 'Planta'}, inplace=True)


6,Planta,Departamento
0,AMALFI,ANTIOQUIA
1,AMERICA,ANTIOQUIA
2,ASNAZU,CAUCA
3,BAYONA,QUINDÍO
4,BELLO,ANTIOQUIA


In [14]:
#Conseguir la información de produccion por departamento
lista_df_produccion = []
for archivo in os.listdir(ruta_produccion):
    if archivo.startswith('produccion') and archivo.endswith('.xlsx'):

        print(f"Archivo válido encontrado, procesando: {archivo}")
        ruta_completa = os.path.join(ruta_produccion, archivo)

        df_temporal = pd.read_excel(ruta_completa, header=2)

        año = re.search(r'(\d{4})', archivo).group(1)
        df_temporal['Año'] = año

        lista_df_produccion.append(df_temporal)

df_produccion_total = pd.concat(lista_df_produccion, ignore_index=True)

print("\n¡Proceso completado!")
print(f"Se procesaron {len(lista_df_produccion)} archivos de producción.")
print(df_produccion_total.head())



Archivo válido encontrado, procesando: produccion2017.xlsx
Archivo válido encontrado, procesando: produccion2020.xlsx
Archivo válido encontrado, procesando: produccion2021.xlsx
Archivo válido encontrado, procesando: produccion2016.xlsx
Archivo válido encontrado, procesando: produccion2022.xlsx
Archivo válido encontrado, procesando: produccion2019.xlsx
Archivo válido encontrado, procesando: produccion2023.xlsx
Archivo válido encontrado, procesando: produccion2018.xlsx
Archivo válido encontrado, procesando: produccion2015.xlsx
Archivo válido encontrado, procesando: produccion2024.xlsx

¡Proceso completado!
Se procesaron 10 archivos de producción.
            Planta Tipo Despacho  Generación [GWh]  Part.[%]   Año
0           AMALFI            ND               0.0       0.0  2017
1  PIEDRAS BLANCAS            ND               0.0       0.0  2017
2     PURIFICACION            ND               0.0       0.0  2017
3        RIO ABAJO            ND               0.0       0.0  2017
4      RIO I

In [15]:
#El dataframe filtrado será
produccion_dpto = df_produccion_total[['Año','Planta','Generación [GWh]']]
print(produccion_dpto.head())
print(produccion_dpto.shape)

    Año           Planta  Generación [GWh]
0  2017           AMALFI               0.0
1  2017  PIEDRAS BLANCAS               0.0
2  2017     PURIFICACION               0.0
3  2017        RIO ABAJO               0.0
4  2017      RIO INGENIO               0.0
(3466, 3)


In [16]:
print(diccionario_plantas.head(10))
print(len(diccionario_plantas))

6                       Planta     Departamento
0                       AMALFI        ANTIOQUIA
1                      AMERICA        ANTIOQUIA
2                       ASNAZU            CAUCA
3                       BAYONA          QUINDÍO
4                        BELLO        ANTIOQUIA
5          CAMPESTRE (CALARCA)          QUINDÍO
6              CAMPESTRE (EPM)        ANTIOQUIA
7                    EL COCUYO  VALLE DEL CAUCA
8                     GUACAICA           CALDAS
9                   INTERMEDIA           CALDAS
314


In [23]:
#Asignar estas plantas manualmente hasta masomenos llegar a un estimativo razonable

plantas_nuevas = ['PAGUA', 'GUATRON', 'ALBAN', 'FLORES 4B', 'TERMOSIERRAB', 'FLORES 1']
departamentos_nuevas = ['CUNDINAMARCA', 'ANTIOQUIA', 'VALLE DEL CAUCA', 'ATLÁNTICO', 'ANTIOQUIA', 'ATLÁNTICO']
plantas_nuevas_dict = {'Planta': plantas_nuevas, 'Departamento': departamentos_nuevas}
df_diccionario = pd.DataFrame(plantas_nuevas_dict)

#Concatenar los resultados al diccionario por dpto
diccionario_plantas = pd.concat([diccionario_plantas, df_diccionario], ignore_index=True)

In [24]:
def crear_llave_normalizada(nombre_planta):
    if not isinstance(nombre_planta, str):
        return ""
    texto = nombre_planta.lower()
    texto = texto.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    palabras_a_quitar = ['planta', 'central', 'sa', 'esp', 'cc', 'sas', 'ltda', '1', '2', 'autog', 'autg', 'agpe']
    for palabra in palabras_a_quitar:
        texto = re.sub(r'\b' + palabra + r'\b', '', texto)

    texto = re.sub(r'\s+', '', texto)
    return texto.strip()

produccion_dpto['llave_planta'] = produccion_dpto['Planta'].apply(crear_llave_normalizada)
diccionario_plantas['llave_planta'] = diccionario_plantas['Planta'].apply(crear_llave_normalizada)

lista_plantas_diccionario = diccionario_plantas['llave_planta'].unique()
plantas_no_encontradas = produccion_dpto[~produccion_dpto['llave_planta'].isin(lista_plantas_diccionario)]

ranking_faltantes = plantas_no_encontradas.groupby('Planta')['Generación [GWh]'].sum().reset_index()
ranking_faltantes_ordenado = ranking_faltantes.sort_values(by='Generación [GWh]', ascending=False)

print(f'Plantas no encontradas {plantas_no_encontradas.shape}')
print("--- Ranking de Plantas Faltantes por Impacto (Top 20) ---")
print(ranking_faltantes_ordenado.head(20))

Plantas no encontradas (1273, 4)
--- Ranking de Plantas Faltantes por Impacto (Top 20) ---
                          Planta  Generación [GWh]
548                   EL MORRO 1        937.470253
542                     CIMARRON        851.053237
549                   EL MORRO 2        825.937034
564             JEPIRACHI 1 - 15        373.227925
575      PARQUE EOLICO GUAJIRA I        213.346203
602             TRINA-VATIA BSLI        189.875826
603            TRINA-VATIA BSLII        177.441325
604           TRINA-VATIA BSLIII        174.781883
569                    LA SIERPE        144.854612
576         PARQUE EÓLICO WESP01        139.530813
543       COGENERADOR COLTEJER 1        118.982442
594                        SINCE        107.105219
528         CELSIA SOLAR BOLIVAR         91.974119
531          CELSIA SOLAR DULIMA         87.179215
553         GRANJA SOLAR FLANDES         83.813653
570                     LA TOLUA         78.761940
508        AUTOG ECOPETROL ORITO         7

/tmp/ipykernel_15503/4117111255.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produccion_dpto['llave_planta'] = produccion_dpto['Planta'].apply(crear_llave_normalizada)


In [25]:
#Cálculo del Porcentaje Asignado
generacion_total = produccion_dpto['Generación [GWh]'].sum()
generacion_no_asignada = plantas_no_encontradas['Generación [GWh]'].sum()
porcentaje_no_asignado = (generacion_no_asignada / generacion_total) * 100

print(f"Hay {len(plantas_no_encontradas)} plantas que no fueron asignadas a un dpto")
print(f"Porcentaje de energía NO ASIGNADA: {porcentaje_no_asignado:.2f}%")
print(f"Porcentaje de energía que SÍ FUE ASIGNADA a un departamento: {100 - porcentaje_no_asignado:.2f}%")

Hay 1273 plantas que no fueron asignadas a un dpto
Porcentaje de energía NO ASIGNADA: 0.87%
Porcentaje de energía que SÍ FUE ASIGNADA a un departamento: 99.13%


In [26]:
ranking_faltantes_ordenado[:21]

,Planta,Generación [GWh]
548,EL MORRO 1,937.470253
542,CIMARRON,851.053237
549,EL MORRO 2,825.937034
564,JEPIRACHI 1 - 15,373.227925
575,PARQUE EOLICO GUAJIRA I,213.346203
602,TRINA-VATIA BSLI,189.875826
603,TRINA-VATIA BSLII,177.441325
604,TRINA-VATIA BSLIII,174.781883
569,LA SIERPE,144.854612
576,PARQUE EÓLICO WESP01,139.530813


In [27]:
# Unimos la tabla de producción con el diccionario usando la llave normalizada.
produccion_con_dpto = pd.merge(
    produccion_dpto,
    diccionario_plantas,
    on='llave_planta',
    how='left'
)

produccion_con_dpto['Departamento'].fillna('No Asignado / Menores', inplace=True)

produccion_agrupada = produccion_con_dpto.groupby(['Año', 'Departamento'])['Generación [GWh]'].sum().reset_index()

produccion_final = produccion_agrupada.rename(columns={
    'Departamento': 'Departamento',
    'Generación [GWh]': 'Produccion_GWh'
})

print("DataFrame Final: Producción por Año y Departamento")
print(produccion_final.head(10))

DataFrame Final: Producción por Año y Departamento
    Año  Departamento  Produccion_GWh
0  2015     ANTIOQUIA    23974.773169
1  2015     ATLÁNTICO    11224.281977
2  2015  BOGOTÁ, D.C.       20.204581
3  2015       BOLÍVAR     1205.155267
4  2015        BOYACÁ     7394.486413
5  2015        CALDAS     1568.395504
6  2015      CASANARE      818.884378
7  2015         CAUCA     1000.681798
8  2015  CUNDINAMARCA    15270.937250
9  2015       CÓRDOBA     1440.367882


/tmp/ipykernel_15503/1777290254.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  produccion_con_dpto['Departamento'].fillna('No Asignado / Menores', inplace=True)


The error `IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/archivos energeticos/demanda_energetica'` occurs because you are trying to use `pd.read_excel` on a directory path (`ruta_demanda`) instead of a file path. Inside the loop, you should be reading each individual Excel file found in the directory. Also, you are currently appending the temporary dataframes to `lista_df_produccion` instead of `lista_df_demanda`.

I will provide a corrected version of the code for cell `huk8Hqx-0BaO` to fix these issues.

In [28]:
#Leer y agrupar la demanda energetica
lista_df_demanda = []
for archivo in os.listdir(ruta_demanda):
    if archivo.startswith('demanda') and archivo.endswith('.csv'):

        print(f"Archivo válido encontrado, procesando: {archivo}")
        ruta_completa = os.path.join(ruta_demanda, archivo)

        df_temporal = pd.read_csv(ruta_completa)

        año = re.search(r'(\d{4})', archivo).group(1)
        df_temporal['Año'] = año

        lista_df_demanda.append(df_temporal)

df_demanda_total = pd.concat(lista_df_demanda, ignore_index=True)

print("\n¡Proceso completado!")
print(f"Se procesaron {len(lista_df_demanda)} archivos de demanda.")
display(df_demanda_total.head())

Archivo válido encontrado, procesando: demanda2021.csv
Archivo válido encontrado, procesando: demanda2018.csv
Archivo válido encontrado, procesando: demanda2023.csv
Archivo válido encontrado, procesando: demanda2024.csv
Archivo válido encontrado, procesando: demanda2022.csv
Archivo válido encontrado, procesando: demanda2015.csv
Archivo válido encontrado, procesando: demanda2016.csv
Archivo válido encontrado, procesando: demanda2019.csv
Archivo válido encontrado, procesando: demanda2020.csv
Archivo válido encontrado, procesando: demanda2017.csv

¡Proceso completado!
Se procesaron 10 archivos de demanda.


,Departamento,Municipio,Demanda Real [MWh],Participación [%],Año
0,Bogotá. D.C.,Bogota D.C,2.270312e+06,"9,99%",2021
1,Bolívar,Cartagena,1.204541e+06,"5,30%",2021
2,Córdoba,Montelibano,1.158674e+06,"5,10%",2021
3,Atlántico,Barranquilla,9.708808e+05,"4,27%",2021
4,Valle Del Cauca,Cali,9.178343e+05,"4,04%",2021


In [29]:
demanda_limpia = df_demanda_total[['Año', 'Departamento', 'Demanda Real [MWh]']].copy()

demanda_limpia.rename(columns={'Demanda Real [MWh]': 'Energia_GWh'}, inplace=True)

demanda_limpia['Tipo'] = 'Demanda'

print("DataFrame de Demanda Limpio y Estandarizado")
display(demanda_limpia.head())

DataFrame de Demanda Limpio y Estandarizado


,Año,Departamento,Energia_GWh,Tipo
0,2021,Bogotá. D.C.,2.270312e+06,Demanda
1,2021,Bolívar,1.204541e+06,Demanda
2,2021,Córdoba,1.158674e+06,Demanda
3,2021,Atlántico,9.708808e+05,Demanda
4,2021,Valle Del Cauca,9.178343e+05,Demanda


In [30]:
demanda_limpia['Departamento'].unique()

array(['Bogotá. D.C.', 'Bolívar', 'Córdoba', 'Atlántico',
       'Valle Del Cauca', 'Antioquia', 'Boyacá', 'Santander', 'Meta',
       'Arauca', 'Cundinamarca', 'Caldas', 'Tolima', 'Norte De Santander',
       'Magdalena', 'La Guajira', 'Risaralda', 'Cauca', 'Huila', 'Cesar',
       'Sucre', 'Casanare', 'Quindío', 'Nariño', 'Putumayo', 'Chocó',
       'Caquetá'], dtype=object)

In [31]:
produccion_para_unir = produccion_final.copy()

produccion_para_unir.rename(columns={'Produccion_GWh': 'Energia_GWh'}, inplace=True)

produccion_para_unir['Tipo'] = 'Produccion'

print("--- DataFrame de Producción listo para la unión ---")
display(produccion_para_unir.head())

--- DataFrame de Producción listo para la unión ---


,Año,Departamento,Energia_GWh,Tipo
0,2015,ANTIOQUIA,23974.773169,Produccion
1,2015,ATLÁNTICO,11224.281977,Produccion
2,2015,"BOGOTÁ, D.C.",20.204581,Produccion
3,2015,BOLÍVAR,1205.155267,Produccion
4,2015,BOYACÁ,7394.486413,Produccion


In [32]:
# Unir los dos DataFrames en una única tabla
df_unificado = pd.concat([produccion_para_unir, demanda_limpia], ignore_index=True)

df_unificado.sort_values(by=['Año', 'Departamento'], inplace=True)

# Verificar el resultado final
print("DataFrame Unificado y Listo para Power BI")
display(df_unificado.head())
print("...")
display(df_unificado.tail())

DataFrame Unificado y Listo para Power BI


,Año,Departamento,Energia_GWh,Tipo
0,2015,ANTIOQUIA,23974.773169,Produccion
1,2015,ATLÁNTICO,11224.281977,Produccion
3139,2015,Antioquia,343.684690,Demanda
3140,2015,Antioquia,20732.464590,Demanda
3141,2015,Antioquia,729.334300,Demanda


...


,Año,Departamento,Energia_GWh,Tipo
2405,2024,Valle Del Cauca,335.79444,Demanda
2435,2024,Valle Del Cauca,257.62071,Demanda
2529,2024,Valle Del Cauca,80.24387,Demanda
2538,2024,Valle Del Cauca,56.27750,Demanda
2546,2024,Valle Del Cauca,28.10949,Demanda


In [33]:
#Normalizar la columna de Dpto

df_unificado['Departamento'] = df_unificado['Departamento'].str.title()

print("Departamentos Normalizados")
print(df_unificado['Departamento'].unique())

Departamentos Normalizados
['Antioquia' 'Atlántico' 'Arauca' 'Bogotá, D.C.' 'Bolívar' 'Boyacá'
 'Bogotá. D.C.' 'Caldas' 'Casanare' 'Cauca' 'Cundinamarca' 'Caquetá'
 'Cesar' 'Chocó' 'Córdoba' 'Huila' 'La Guajira' 'Magdalena' 'Meta'
 'Nariño' 'Norte De Santander' 'No Asignado / Menores' 'Putumayo'
 'Quindío' 'Risaralda' 'Santander' 'Sucre' 'Tolima' 'Valle Del Cauca']


In [34]:
df_unificado.head()

,Año,Departamento,Energia_GWh,Tipo
0,2015,Antioquia,23974.773169,Produccion
1,2015,Atlántico,11224.281977,Produccion
3139,2015,Antioquia,343.684690,Demanda
3140,2015,Antioquia,20732.464590,Demanda
3141,2015,Antioquia,729.334300,Demanda


In [ ]:
#Agrupar por las tres columnas y sumar la energía
df_agrupado = df_unificado.groupby(['Año', 'Departamento', 'Tipo'])['Energia_GWh'].sum().reset_index()

print(df_agrupado.head(30))

     Año  Departamento        Tipo   Energia_GWh
0   2015     Antioquia     Demanda  2.875311e+06
1   2015     Antioquia  Produccion  2.397477e+04
2   2015        Arauca     Demanda  3.955137e+05
3   2015     Atlántico     Demanda  1.149667e+06
4   2015     Atlántico  Produccion  1.122428e+04
5   2015  Bogotá, D.C.  Produccion  2.020458e+01
6   2015  Bogotá. D.C.     Demanda  2.764870e+06
7   2015       Bolívar     Demanda  1.107525e+06
8   2015       Bolívar  Produccion  1.205155e+03
9   2015        Boyacá     Demanda  1.900287e+06
10  2015        Boyacá  Produccion  7.394486e+03
11  2015        Caldas     Demanda  4.534544e+05
12  2015        Caldas  Produccion  1.568396e+03
13  2015       Caquetá     Demanda  4.503246e+03
14  2015      Casanare     Demanda  5.502151e+04
15  2015      Casanare  Produccion  8.188844e+02
16  2015         Cauca     Demanda  3.913336e+05
17  2015         Cauca  Produccion  1.000682e+03
18  2015         Cesar     Demanda  2.415887e+05
19  2015         Cho

In [37]:
df_nuevo = df_agrupado.pivot_table(
    index=['Año', 'Departamento'],
    columns='Tipo',
    values='Energia_GWh'
).reset_index()

df_nuevo = df_nuevo.fillna(0)
df_nuevo.columns.name = None
df_nuevo.rename(columns={'Demanda': 'demanda', 'Produccion': 'produccion'}, inplace=True)

print(df_nuevo.head(20))

     Año  Departamento       demanda    produccion
0   2015     Antioquia  2.875311e+06  23974.773169
1   2015        Arauca  3.955137e+05      0.000000
2   2015     Atlántico  1.149667e+06  11224.281977
3   2015  Bogotá, D.C.  0.000000e+00     20.204581
4   2015  Bogotá. D.C.  2.764870e+06      0.000000
5   2015       Bolívar  1.107525e+06   1205.155267
6   2015        Boyacá  1.900287e+06   7394.486413
7   2015        Caldas  4.534544e+05   1568.395504
8   2015       Caquetá  4.503246e+03      0.000000
9   2015      Casanare  5.502151e+04    818.884378
10  2015         Cauca  3.913336e+05   1000.681798
11  2015         Cesar  2.415887e+05      0.000000
12  2015         Chocó  1.127084e+04      0.000000
13  2015  Cundinamarca  1.774214e+06  15270.937250
14  2015       Córdoba  1.470549e+06   1440.367882
15  2015         Huila  2.176769e+05   1733.114951
16  2015    La Guajira  3.121127e+05   2446.342910
17  2015     Magdalena  2.964456e+05      0.000000
18  2015          Meta  6.55797

In [38]:
departamentos_unicos = df_nuevo['Departamento'].unique()

nuevos_departamentos = [
    'Amazonas', 'Guainía', 'Guaviare',
    'San Andrés, Providencia y Santa Catalina', 'Vaupés', 'Vichada'
]
todos_los_deptos = sorted(list(set(list(departamentos_unicos) + nuevos_departamentos)))
todos_los_años = df_nuevo['Año'].unique()

# --- Bucle para encontrar y crear las filas faltantes ---
filas_nuevas = []

for año in todos_los_años:
  deptos_existentes = df_nuevo[df_nuevo['Año'] == año]['Departamento'].unique()

  for depto in todos_los_deptos:
    if depto not in deptos_existentes:
      filas_nuevas.append({'Año': año, 'Departamento': depto, 'demanda': 0, 'produccion': 0})

# --- Creación y concatenación del DataFrame final (tu código aquí es correcto) ---
df_filas_nuevas = pd.DataFrame(filas_nuevas)

df_final_bucle = pd.concat([df_nuevo, df_filas_nuevas], ignore_index=True)

# Imprimir las últimas filas para verificar que se agregaron correctamente
print("Verificando una de las filas agregadas (Amazonas):")
print(df_final_bucle[df_final_bucle['Departamento'] == 'Amazonas'])

Verificando una de las filas agregadas (Amazonas):
      Año Departamento  demanda  produccion
290  2015     Amazonas      0.0         0.0
296  2016     Amazonas      0.0         0.0
302  2017     Amazonas      0.0         0.0
308  2018     Amazonas      0.0         0.0
314  2019     Amazonas      0.0         0.0
320  2020     Amazonas      0.0         0.0
326  2021     Amazonas      0.0         0.0
332  2022     Amazonas      0.0         0.0
338  2023     Amazonas      0.0         0.0
344  2024     Amazonas      0.0         0.0


In [39]:
# Ordenamos el DataFrame final por la columna 'Año' y luego por 'Departamento'
df_ordenado = df_final_bucle.sort_values(by=['Año', 'Departamento']).reset_index(drop=True)

# Mostramos las primeras 35 filas para que puedas ver el orden
print(df_ordenado.head(40))

     Año                              Departamento       demanda    produccion
0   2015                                  Amazonas  0.000000e+00      0.000000
1   2015                                 Antioquia  2.875311e+06  23974.773169
2   2015                                    Arauca  3.955137e+05      0.000000
3   2015                                 Atlántico  1.149667e+06  11224.281977
4   2015                              Bogotá, D.C.  0.000000e+00     20.204581
5   2015                              Bogotá. D.C.  2.764870e+06      0.000000
6   2015                                   Bolívar  1.107525e+06   1205.155267
7   2015                                    Boyacá  1.900287e+06   7394.486413
8   2015                                    Caldas  4.534544e+05   1568.395504
9   2015                                   Caquetá  4.503246e+03      0.000000
10  2015                                  Casanare  5.502151e+04    818.884378
11  2015                                     Cauca  

In [40]:
import os # Importamos la librería 'os' para manejar carpetas

#Definir el nombre de la carpeta de salida
output_dir = 'output'

#Crear la carpeta si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#Definir la ruta completa del archivo de salida
ruta_salida = os.path.join(output_dir, 'datos_filtrados.csv')

#Exportar el DataFrame a CSV
df_ordenado.to_csv(ruta_salida, index=False)

print(f"¡DataFrame exportado exitosamente a '{ruta_salida}'!")

¡DataFrame exportado exitosamente a 'output/datos_filtrados.csv'!
